## Properties Data from Houm

La idea es obtener georeferencias de distintas propiedas en Pudahuel (filtro en la parte de comuna) que estén en arriendo

Con esos datos, poblar una tabla para tener de muestra.


### "similar-properties"

Luego de revisar la página, encontramos una api que permite obtener datos de propiedades similares, entregando un "id" de propiedad.

Esta es la estructura:

**https://api.arriendoasegurado.com/properties/{id}/similar-properties?mode=rental&n_similar={num}&n_photos={num}**

Al hacer un get, devuelve una lista con un json con muchos datos sobre una propiedad, entre los cuales tenemos las coordenadas de una propiedad.

La usaremos para obtener los datos buscados

Vamos a probarla

In [96]:
id_prop = '57412' 

url_houm = f'https://api.arriendoasegurado.com/properties/{id_prop}/similar-properties?mode=rental&n_similar=50&n_photos=5'

res = requests.get(url_houm)

prop = res.json()[0]

print(prop['id'], prop['property_details'][0]['latitud'], prop['property_details'][0]['longitud'])

6489 -33.4538792 -70.6544325


In [7]:
greatest_dict = {}

for prop in res.json():
    
    if prop['id'] in greatest_dict:
        continue
    else:
        
        greatest_dict[prop['id']] = {"lat": prop['property_details'][0]['latitud'],
                                    "lon": prop['property_details'][0]['latitud']}
greatest_dict

**Es perfecto**

Para cumplir con el propósito de sacar algunos datos de muestra. 
Crearemos algunas funciones que permitan extraer los datos y finalmente los guardaremos en un archivo

In [82]:
def get_similar_properties(actual_prop: dict, id_prop: str) -> dict:
   
    properties = {}
    url_houm = f'https://api.arriendoasegurado.com/properties/{id_prop}/similar-properties?mode=rental&n_similar=100&n_photos=1'
    response = requests.get(url_houm)
    if response.status_code == 200:
        
        for prop in response.json():
            
            if prop['id'] not in actual_prop:
                
                properties[prop['id']] = {"lat": prop['property_details'][0]['latitud'],
                                          "lon": prop['property_details'][0]['longitud']}
    return properties

def get_properties(n: int, id_prop: str) -> dict:
    
    properties = {}
    visited_id = []
    while len(list(properties.keys())) < n:
        actual_value = len(list(properties.keys()))
        print(actual_value)
        if actual_value == 0:
            new_properties = get_similar_properties(properties, id_prop)
            visited_id.append(id_prop)
            properties.update(new_properties)
                    
        else:
            keys = list(properties.keys())
            new_ids = [id_prop for id_prop in keys if id_prop not in visited_id]
            for id_prop in new_ids:
                new_properties = get_similar_properties(properties, id_prop)
                visited_id.append(id_prop)
                properties.update(new_properties)
        new_value = len(list(properties.keys()))
        if actual_value == new_value:
            print(f"El máximo de propiedades similares se ha alcanzado en {new_value}. No hay más propiedades similares")
            break
    return properties

def dict_properties_to_list_properties(properties_dict: dict) -> list:
    
    properties_list = []
    for k, v in json_properties.items():

        tmp_properties_dict = {
            "property_id": k,
            "lat": v['lat'],
            "lon": v['lon']
        }
        properties_list.append(tmp_properties_dict)
    
    return properties_list 

In [88]:
properties_list_starting_with_57412 = dict_properties_to_list_properties(get_properties(200, '57412'))
file=open("properties_list.txt","w+")
for prop in properties_list_starting_with_57412:
    file.write(f'{prop}\n')
file.close()

[{'property_id': 6489, 'lat': -33.4538792, 'lon': -70.6544325},
 {'property_id': 20159, 'lat': -33.4407471, 'lon': -70.6926797},
 {'property_id': 40530, 'lat': -33.4534707, 'lon': -70.7055654},
 {'property_id': 45300, 'lat': -33.4517244, 'lon': -70.696716},
 {'property_id': 54186, 'lat': -33.4394842, 'lon': -70.6620545},
 {'property_id': 55121, 'lat': -33.452548, 'lon': -70.705951},
 {'property_id': 55346, 'lat': -33.454297, 'lon': -70.6558766},
 {'property_id': 55394, 'lat': -33.4505712, 'lon': -70.6963284},
 {'property_id': 55997, 'lat': -33.4397017, 'lon': -70.6602475},
 {'property_id': 56353, 'lat': -33.4576973, 'lon': -70.6704505},
 {'property_id': 56727, 'lat': -33.4532628, 'lon': -70.6993994},
 {'property_id': 57055, 'lat': -33.4481269, 'lon': -70.6926089},
 {'property_id': 57073, 'lat': -33.4550015, 'lon': -70.7038037},
 {'property_id': 57462, 'lat': -33.4535687, 'lon': -70.6955127},
 {'property_id': 57463, 'lat': -33.4535687, 'lon': -70.6955127},
 {'property_id': 57908, 'lat':

Listo, con eso se muestra el como se obtuvo los datos de muestra.

Los insertaremos en la bbdd usando el servicio REST creado.

In [98]:
from tqdm import tqdm
for prop in properties_list_starting_with_57412:
    property_id = prop["property_id"]
    data = {"lat": prop["lat"], "lon": prop["lon"]}
    url = f"http://localhost:8080/properties/{property_id}/insert-markers"
    requests.post(url, json=data)
    res = requests.post(url, json=prop)